In [1]:
import cobra
from cobra.sampling import sample
from cobra.sampling import OptGPSampler
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn as sns
import os
from cameo import fba
import pickle
import lzma

In [2]:
def create_rxn(model, name, cpd_dict, direction="both", limits=None):
    # Create a reaction from a dict with {"CPD1": -1, "CPD2: 1"}
    # Assuming the cpd ids are already in the model
    if direction not in ["forward", "reverse", "both"]:
        raise ValueError("Direction must be either forward, reverse, or both")
    rxn = cobra.Reaction(name)
    if limits is not None:
        if len(limits) != 2:
            raise ValueError("Limits must be of length 2")
        else:
            rxn.lower_bound = limits[0]
            rxn.upper_bound = limits[1]
    elif direction == "both":
        rxn.lower_bound = -1000
        rxn.upper_bound = 1000
    elif direction == "forward":
        rxn.lower_bound = 0
        rxn.upper_bound = 1000
    else:
        rxn.lower_bound = -1000
        rxn.upper_bound = 0
    rxn.add_metabolites({
        getattr(model.metabolites, cpd): val for cpd, val in cpd_dict.items()
    })
    print("Adding the following reaction to model:")
    print(rxn)
    model.add_reactions([rxn])
def multisample(model, n, runs=5, threads=1):
    n_per_run = int(n/runs)
    if n_per_run*runs != n:
        print("Note: specified number of samples is not divisible by number of runs")
    print("Collecting {r} samples of size {x}, for {y} total samples".format(r=runs, x=n_per_run, y=runs*n_per_run))
    #p = Pool(nodes=threads)
    #result = p.uimap(partial(sample, processes=1), (model,)*runs, (n_per_run,)*runs)
    #p.close()
    #p.join()
    #p.clear()
    result = []
    for i in range(runs):
        s = sample(model, n_per_run, processes=threads)
        result.append(s)
    return pd.concat(result, axis=0).reset_index()
def add_fixed_constraint(model, rxn, value):
    """
    model = cobra formatted model
    rxn = string of reaction id
    value = a single int/float to constrain the reaction to, or a list/tuple like (lb, ub)
    """
    try:
        lower = value[0]
        upper = value[1]
    except (TypeError, IndexError):
        lower = value
        upper = value
    constraint = model.problem.Constraint(
        getattr(model.reactions, rxn).flux_expression,
        lb=lower, ub=upper)
    model.add_cons_vars(constraint)
    return constraint

### Preparation
The code below assumes:
1. The code is being run from the GEM-iCbes/manuscript directory

In [3]:
cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"

### SBML file generation
SBML files were generated from psamm using the `psamm-model sbmlexport` command.

In the open model, the following constraints were added to limits.yaml:
```
- reaction: R00700
  lower: 0
  upper: 0
- reaction: R07181
  lower: 0
  upper: 0
```

In the closed model, the constraints were as follows:
```
- reaction: R00700
  lower: 0
  upper: 1000
- reaction: R07181
  lower: 0
  upper: 1000
```

In [4]:
with open("../sbmls/b971180d_YM_closed.sbml") as f:
    YM_closed = cobra.io.read_sbml_model(f)
with open("../sbmls/b971180d_YM_open.sbml") as f:
    YM_open = cobra.io.read_sbml_model(f)
with open("../sbmls/b971180d_YM_CODH_closed.sbml") as f:
    CODH_closed = cobra.io.read_sbml_model(f)
with open("../sbmls/b971180d_YM_CODH_open.sbml") as f:
    CODH_open = cobra.io.read_sbml_model(f)

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.


## Universal constraints
1. GAPDH is removed in glycolytic conditions
2. GAPOR is removed in gluconeogenic conditions

In [5]:
for model in [YM_closed, YM_open, CODH_closed, CODH_open]:
    model.reactions.R01061.upper_bound = 0
    model.reactions.R01063.upper_bound = 0

## Calibrate overexpression for 'open' and 'closed' models
Ethanol production values were collected after 21 hours of growth at 95 C in the *P. furiosus* COM1c and OE-AdhF strains (Lipscomb et al. 2023). The ratio of ethanol production in the control (COM1) strain to the OE-AdhF strain was defined as the `scale` variable below.

Lipscomb GL, Crowley A, Nguyen DMN, Keller MW, O’Quinn H, Tanwee TNN, Vailionis
JL, Zhang K, Zhang Y, Kelly RM, Adams MWW. 2023. Manipulating fermentation
pathways in the hyperthermophilic archaeon Pyrococcus furiosus for ethanol production up
to 95°C driven by carbon monoxide oxidation. Appl Environ Microbiol. under review
(Companion submission).

In [8]:
s1_closed = multisample(YM_closed, 2500, runs=10, threads=5)
s1_open = multisample(YM_open, 2500, runs=10, threads=5)

In [9]:
# derived from experimental ratio of ethanol production at 21 hr in 95C YM for MW004 / MW631tile(s1_closed.TP_ETOH, .25)
scale = 0.2634177489 
OE_lower = np.quantile(s1_closed.TP_ETOH, .25)
cotrol_upper = scale * np.quantile(s1_closed.TP_ETOH, 0.75)
print("Closed model ethanol bounds = {}".format((OE_lower, 1000)))
YM_closed_OE = YM_closed.copy()
YM_closed_OE.reactions.TP_ETOH.lower_bound = OE_lower
s1_YM_closed_OE = multisample(YM_closed_OE, 2500, runs=10, threads=5)

Closed model ethanol bounds = (3.292650740688972, 1000)


In [10]:
# derived from experimental ratio of ethanol production at 21 hr in 95C YM for MW004 / MW631tile(s1_open.TP_ETOH, .25)
scale = 0.2634177489 
OE_lower = np.quantile(s1_open.TP_ETOH, .25)
cotrol_upper = scale * np.quantile(s1_open.TP_ETOH, 0.75)
print("open model ethanol bounds = {}".format((OE_lower, 1000)))
YM_open_OE = YM_open.copy()
YM_open_OE.reactions.TP_ETOH.lower_bound = OE_lower
s1_YM_open_OE = multisample(YM_open_OE, 2500, runs=10, threads=5)

open model ethanol bounds = (0.6934084904161786, 1000)


In [7]:
with lzma.open("pickles/s1.pkl.xz", "wb") as f:
    pickle.dump([s1_closed, s1_open, s1_YM_closed_OE, s1_YM_open_OE], f)

In [6]:
with lzma.open("pickles/s1.pkl.xz", "rb") as f:
    s1_closed, s1_open, s1_YM_closed_OE, s1_YM_open_OE = pickle.load(f)

In [7]:
scale = 0.2634177489 
OE_lower_closed = np.quantile(s1_closed.TP_ETOH, .25)
OE_lower_open = np.quantile(s1_open.TP_ETOH, .25)
control_upper_closed = scale * np.quantile(s1_closed.TP_ETOH, 0.75)
control_upper_open = scale * np.quantile(s1_open.TP_ETOH, 0.75)

print(OE_lower_closed)
print(OE_lower_open)
print(control_upper_closed)
print(control_upper_open)

3.292650740688972
0.6934084904161786
2.4935252567276263
0.8109819796784484


### Creating model files
The sbml model files must be created manually using `psamm-model sbmlexport`, and are required for downstream analyses. To create them apply the following constraints to limits.yaml before exporting:

OE-AdhF in closed system:
```
- reaction: TP_ETOH
  lower: 3.293
  upper: 1000
- reaction: R00700
  lower: 0
  upper: 1000
- reaction: R07181
  lower: 0
  upper: 1000
```
OE-AdhF in open system:
```
- reaction: TP_ETOH
  lower: 0.6934
  upper: 1000
- reaction: R00700
  lower: 0
  upper: 0
- reaction: R07181
  lower: 0
  upper: 0
```
Control (COM1c) in closed system:
```
- reaction: TP_ETOH
  lower: 0
  upper: 2.4935
- reaction: R00700
  lower: 0
  upper: 1000
- reaction: R07181
  lower: 0
  upper: 1000
```
OE-AdhF in open system:
```
- reaction: TP_ETOH
  lower: 0
  upper: 0.8110
- reaction: R00700
  lower: 0
  upper: 0
- reaction: R07181
  lower: 0
  upper: 0
```